In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
np.random.seed(42)
X = np.random.rand(100, 5)
y = np.random.randint(0, 2, 100)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [3]:
num_features = 5
num_particles = 3
max_iter = 10

w = 0.7
c1 = 2
c2 = 2
r1 = 0.5
r2 = 0.3

In [4]:
positions = np.array([
    [1, 0, 1, 0, 1],
    [0, 1, 1, 0, 0],
    [1, 1, 0, 1, 0]
], dtype=int)

In [5]:
velocities = np.array([
    [0.2, -0.1, 0.3, -0.2, 0.1],
    [-0.1, 0.2, 0.1, -0.3, -0.2],
    [0.1, 0.1, -0.2, 0.3, -0.1]
])

In [6]:
def fitness_function(position):
    if np.sum(position) == 0:
        return 0

    selected_features = np.where(position == 1)[0]

    X_train_sel = X_train[:, selected_features]
    X_test_sel = X_test[:, selected_features]

    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_train_sel, y_train)

    y_pred = model.predict(X_test_sel)
    return accuracy_score(y_test, y_pred)

In [7]:
pbest_positions = positions.copy()
pbest_scores = np.array([fitness_function(p) for p in positions])

gbest_index = np.argmax(pbest_scores)
gbest_position = pbest_positions[gbest_index].copy()
gbest_score = pbest_scores[gbest_index]

print("Initial Global Best:", gbest_position, gbest_score)

Initial Global Best: [1 1 0 1 0] 0.5333333333333333


In [8]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [9]:
for iteration in range(1, max_iter + 1):
    print(f"\nIteration {iteration}")

    for i in range(num_particles):
        velocities[i] = (
            w * velocities[i]
            + c1 * r1 * (pbest_positions[i] - positions[i])
            + c2 * r2 * (gbest_position - positions[i])
        )

        probs = sigmoid(velocities[i])
        positions[i] = (probs >= 0.5).astype(int)

        score = fitness_function(positions[i])

        if score > pbest_scores[i]:
            pbest_scores[i] = score
            pbest_positions[i] = positions[i].copy()

    best_particle = np.argmax(pbest_scores)
    if pbest_scores[best_particle] > gbest_score:
        gbest_score = pbest_scores[best_particle]
        gbest_position = pbest_positions[best_particle].copy()

    print("Global Best:", gbest_position, gbest_score)


Iteration 1
Global Best: [1 1 0 1 0] 0.5333333333333333

Iteration 2
Global Best: [1 1 0 1 0] 0.5333333333333333

Iteration 3
Global Best: [1 1 0 1 0] 0.5333333333333333

Iteration 4
Global Best: [1 1 0 1 0] 0.5333333333333333

Iteration 5
Global Best: [1 1 0 1 0] 0.5333333333333333

Iteration 6
Global Best: [1 1 0 1 0] 0.5333333333333333

Iteration 7
Global Best: [1 1 0 1 0] 0.5333333333333333

Iteration 8
Global Best: [1 1 0 1 0] 0.5333333333333333

Iteration 9
Global Best: [1 1 0 1 0] 0.5333333333333333

Iteration 10
Global Best: [1 1 0 1 0] 0.5333333333333333


In [10]:
print("\nFinal Global Best Feature Subset:", gbest_position)
print("Final Accuracy:", gbest_score)

selected_features = np.where(gbest_position == 1)[0] + 1
print("Selected Features:", selected_features)


Final Global Best Feature Subset: [1 1 0 1 0]
Final Accuracy: 0.5333333333333333
Selected Features: [1 2 4]
